In [1]:
import os
import numpy as np
import pandas as pd

## movie_genre

In [ ]:
current_dir = os.getcwd()
movies_data = pd.read_csv(current_dir + '/data/movies_data.csv')

In [ ]:
movies_data = movies_data[['genre1','genre2','genre3']]

In [ ]:
movies_data

In [ ]:
import pandas._testing as tm


def unpivot(frame):
    N, K = frame.shape
    data = {
        "genre": np.asarray(frame.to_numpy()).ravel("F"),
        "movie_id": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["movie_id", "genre"])


df = unpivot(movies_data)

In [ ]:
sorted_df = df.sort_values(by='movie_id').reset_index(drop=True)

In [ ]:
sorted_df

In [ ]:
genre = pd.read_csv(current_dir + '/data/genre.csv')

In [ ]:
genre.rename(columns={'Unnamed: 0': 'genre_id'}, inplace=True)

In [ ]:
df_left_join = pd.merge(sorted_df, genre, left_on='genre', right_on='genre', how='left')
df_left_join

In [ ]:
df_left_join = df_left_join.dropna()

In [ ]:
df_left_join['genre_id'] = df_left_join['genre_id'].astype(int)

In [ ]:
movie_genre = df_left_join[['movie_id', 'genre_id']]

In [ ]:
movie_genre.to_csv(current_dir+'/data/movie_genre.csv', index=False)

## genre_keyword

In [ ]:
genre

In [ ]:
genre_value = ['액션','액션', '액션', '코미디', '코미디', '다큐멘터리', '판타지', '판타지', '공포', '음악', '로맨스','로맨스', '스포츠', '서부', 'Made in Europe', '애니메이션', '범죄', '드라마', '역사', '가족', '스릴러', 'SF','SF','SF', '전쟁',  '전쟁', 'Reality TV']
keyword = ['권선징악', '긴장', '음직임', '웃음', '유머', '일상', '환상', '상상', '귀신', '뮤지컬', '사랑','연애', '운동', '서부', '프랑스', '만화', '살인', '드라마', '조선', '부모님', '스릴러', '외계인','우주','중력', '군대', '죽음', '실제']

In [ ]:
genre_keyword = pd.DataFrame(columns=['장르', 'keyword'])

In [ ]:
genre_keyword['장르'] = genre_value
genre_keyword['keyword'] = keyword

In [ ]:
genre_keyword

In [ ]:
df_left_join = pd.merge(genre_keyword, genre, left_on='장르', right_on='genre', how='left')


In [ ]:
df_left_join[['genre_id','keyword']].to_csv(current_dir + '/data/genre_keyword_sample.csv', index=False)

## movie_summary

In [2]:
current_dir = os.getcwd()
movies = pd.read_csv(current_dir + '/data/movies_data.csv')
movie_genre = pd.read_csv(current_dir + '/data/movie_genre.csv')

In [3]:
movies.columns

Index(['Unnamed: 0', 'title', 'type', 'open_year', 'rate', 'running_time',
       'genre1', 'genre2', 'genre3', 'actor', 'director', 'summary'],
      dtype='object')

In [4]:
movies.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)
movies_summary = movies[['movie_id', 'summary']]

In [5]:
join_table = pd.merge(movie_genre, movies_summary, on='movie_id', how='left')

In [6]:
genre_summary = pd.DataFrame(join_table.groupby(['genre_id'])['summary'].sum().reset_index())

In [7]:
drop_idx = join_table[join_table['summary'].str.contains(r'e')].index
join_table.drop(drop_idx, inplace=True)

In [8]:
get_30 = lambda x: x[:30]

In [9]:
grouped = join_table.groupby(['genre_id'])
grouped.size()

genre_id
0     412
1     484
2      57
3     223
4     161
5      53
6     298
7      32
8      18
10    108
11    247
12    849
13     79
14    107
15    518
16    174
17     46
dtype: int64

> genre_id 9(Made in Europe), 18(Reality TV) 없음

In [10]:
selected_summary = grouped.apply(get_30).reset_index(drop=True)

In [11]:
genre_summary = selected_summary[['genre_id', 'summary']]
genre_summary.head()

,genre_id,summary
0,0,유엔 가입을 위해 다수의 투표권을 지닌 아프리카 대륙에서 열심히 외교 활동을 벌이던...
1,0,진실을 위해서라면 몸을 사리지 않고 사회의 부조리를 취재하는 정의로운 열혈 기자 에...
2,0,평범하고 내성적인 고등학생 피터 파커는 우연히 방사능에 감염된 거미에 물린다.그 후...
3,0,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...
4,0,토니 스타크의 제안을 받고 시빌 워에 참여한 피터 파커는 다음 임무가 언제일지 설렘...


In [12]:
genre_summary.shape

(498, 2)

In [17]:
genre_summary = pd.DataFrame(genre_summary.groupby(['genre_id'])['summary'].sum().reset_index(drop=True))

In [18]:
genre_summary.head()

,summary
0,유엔 가입을 위해 다수의 투표권을 지닌 아프리카 대륙에서 열심히 외교 활동을 벌이던...
1,평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게...
2,"권력의 정점에 서고픈 검사, 무소불위 권력을 쥐고 폼 나게 살고 싶었던 태수는 우여..."
3,평범하고 내성적인 고등학생 피터 파커는 우연히 방사능에 감염된 거미에 물린다.그 후...
4,30일마다 랜덤으로 레벨이 바뀌는 극한 생존의 수직 감옥 플랫폼. 최상위 레벨 0에...


In [33]:
len(genre_summary.iloc[0].values[0])

8132

### summary에서 keyword 추출

In [34]:
import os
import numpy as np 
import pandas as pd 

import tweepy
from konlpy.tag import Okt, Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
from hanspell import spell_checker

from pprint import pprint
import re
from math import log # IDF 계산을 위해

import warnings
warnings.filterwarnings(action='ignore')

In [35]:
genre_df = pd.read_csv(current_dir + '/data/genre_df.csv')

In [36]:
# 불용어 설정
stop_words = set(genre_df['genre'])
stop_words.add('영화')

def extract_kor(doc):
    pattern = re.compile(' [^ㄱ-ㅣ가-힣.]') # 한글만 추출
    kor_doc = pattern.sub('', doc)

    # 네이버 맞춤법 검사기는 한번에 최대 500자만 교정할 수 있기 때문에 장문의 글은 리스트로 나눠 요청해야 한다.
    input_convert = kor_doc.replace('.', '#').split('#')
 
    input_list =  [""]
    
    for i in input_convert:
        if len(input_list[-1]) + len(i) < 500:
            input_list[-1] += i
        else:
            input_list.append(i)  
            
    
    docs = spell_checker.check(input_list)
    result = ' '.join([doc.checked for doc in docs])

    return result

def tokenizing(text):
    tokenizer = Mecab()
    total_tokens = tokenizer.nouns(text)
    total_tokens = [token for token in total_tokens if token not in stop_words and len(token) > 1]
    return ' '.join(total_tokens)
    
def preprocessing(corpus):
    kor_corpus = [extract_kor(doc) for doc in corpus]
    # print(kor_corpus)
    kor_tokens = [tokenizing(doc) for doc in kor_corpus]
    return kor_tokens

In [37]:
contents = list(genre_summary['summary'])
kor_tokens = preprocessing(contents)

kor_total_tokens = []
for L in kor_tokens:
    kor_total_tokens += L.split()

In [39]:
vocab = list(set(kor_total_tokens))
vocab.sort()

In [40]:
# 총 문서의 수
N = len(kor_tokens) 

def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in kor_tokens:
    df += t in doc
  return log(N/(df+1))

def tfidf(t, d):
  return tf(t,d)* idf(t)

In [41]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([])
  d = kor_tokens[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_.head()

,가게,가난,가늠,가능,가담,가동,가드,가디언,가르,가리봉동,...,히브리,히스,히어로,히트,히트송,히틀러,힌디어,힐다,힐러리,힙합
0,0,0,0,2,0,1,0,0,0,1,...,0,0,4,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,1,0,0,0,0,0,0,...,0,0,2,0,0,1,0,0,0,0
4,0,0,0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_.head()

,IDF
가게,0.635989
가난,0.887303
가늠,2.140066
가능,0.435318
가담,1.446919


In [43]:
result = []
for i in range(N):
  result.append([])
  d = kor_tokens[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_.head()

,가게,가난,가늠,가능,가담,가동,가드,가디언,가르,가리봉동,...,히브리,히스,히어로,히트,히트송,히틀러,힌디어,힐다,힐러리,힙합
0,0.000000,0.000000,0.0,0.870636,0.000000,1.223775,0.000000,0.0,0.0,1.734601,...,0.0,0.0,4.895102,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,3.671326,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.734601,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,1.907966,0.887303,0.0,0.435318,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,2.447551,0.0,0.0,1.734601,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.870636,1.446919,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [44]:
df = tfidf_.transpose()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
가게,0.000000,0.0,0.0,1.907966,0.000000,0.635989,1.271978,0.000000,0.635989,0.000000,0.635989,0.635989,0.000000,1.907966,0.635989,0.000000,0.000000
가난,0.000000,0.0,0.0,0.887303,0.000000,0.000000,2.661910,1.774606,0.000000,1.774606,0.000000,0.000000,0.887303,0.000000,0.000000,0.887303,0.000000
가늠,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.140066,0.000000,0.000000,0.000000,0.000000
가능,0.870636,0.0,0.0,0.435318,0.870636,0.435318,0.000000,1.305954,0.000000,0.870636,0.000000,0.435318,0.000000,0.000000,0.435318,1.305954,0.870636
가담,0.000000,0.0,0.0,0.000000,1.446919,0.000000,0.000000,0.000000,1.446919,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.446919


In [45]:
df.columns = ['액션', '코미디', '다큐멘터리', '판타지', '공포', '음악', '로맨스', '스포츠', '서부','애니메이션', '범죄', '드라마', '역사', '가족', '스릴러', 'SF', '전쟁']

In [48]:
for genre in df.columns:
    print('========= %s =========' % genre)
    print(df[genre].sort_values(ascending=False).head(5))
    print(df[df[genre]>3][genre].sort_values(ascending=False))

========= 액션 =========
스파이더     19.787624
스파이더맨    16.663262
외계       12.497446
스파이      10.081937
피터        9.141679
Name: 액션, dtype: float64
스파이더     19.787624
스파이더맨    16.663262
외계       12.497446
스파이      10.081937
피터        9.141679
           ...    
사관        3.124362
유령        3.124362
복판        3.124362
북한        3.124362
수배        3.124362
Name: 액션, Length: 67, dtype: float64
========= 코미디 =========
스파이더     11.455993
귀도       10.700331
애슐리       8.673005
조수아       8.560265
스파이더맨     8.331631
Name: 코미디, dtype: float64
스파이더     11.455993
귀도       10.700331
애슐리       8.673005
조수아       8.560265
스파이더맨     8.331631
미아        7.234595
이사벨       6.938404
스파이       5.836911
클레어       5.787676
세바스찬      5.787676
마일스       5.787676
르네        5.203803
잉그리드      5.203803
바스        4.895102
마일        4.895102
패밀리       4.895102
혜성        4.895102
매튜        4.340757
연주        4.340757
유령        4.165815
피터        3.917863
크리스마스     3.815933
테오        3.671326
충돌        3.671326
히어로       

## 형용사 키워드 추출

In [52]:
Mecab().tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NNB': '의존 명사',
 'NNBC': '단위를 나타내는 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'SC': '구분자 , · / :',
 'SE': '줄임표 …',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SSC': '닫는 괄호 ), ]',
 'SSO': '여는 괄호 (, [',
 'SY': '기타 기호',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

In [117]:

FEATURE_POS = ['VA+ETM']# ["Noun", "Verb", "Adverb", "Adjective"]

def text_tokenizing(doc):
    documents = []
    tokenizer = Mecab()
    text_pos = [pair for pair in tokenizer.pos(doc) if pair[0] not in stop_words and len(pair[0]) > 1]
    words = []
    # return pprint(text_pos)
    
    for word, pos in text_pos:
        if pos in FEATURE_POS:
            words.append(word)

    total_words = ' '.join(words)
    return total_words

In [120]:
    
def va_preprocessing(corpus):
    kor_corpus = [extract_kor(doc) for doc in corpus]
    # print(kor_corpus)
    ko_tokens = [text_tokenizing(doc) for doc in contents]
    ko_tokens = ''.join(ko_tokens)
    return kor_tokens

In [124]:
ko_tokens = [''.join(ko_tokens) for token in ko_tokens]

In [131]:
for L in kor_tokens:
    kor_total_tokens += L

In [133]:
vocab = list(set(kor_total_tokens))
vocab.sort()

In [134]:
result = []
N = len(kor_tokens) 

tf_ = pd.DataFrame(result, columns = vocab)
tf_.head()
# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([])
  d = kor_tokens[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_.head()

,가쁜,감미로운,강한,거센,거친,귀한,급한,기나긴,껄끄러운,끈질긴,...,즐거운,차가운,차디찬,착한,촌스러운,친한,커다란,파란,푸른,힘든
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"])
idf_.head()

,IDF
가쁜,2.140066
감미로운,1.446919
강한,0.435318
거센,0.887303
거친,1.446919


In [136]:
result = []
for i in range(N):
  result.append([])
  d = kor_tokens[i]
  for j in range(len(vocab)):
    t = vocab[j]
    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)

In [137]:
df = tfidf_.transpose()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
가쁜,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.140066,0.000000
감미로운,0.0,1.446919,0.0,0.000000,0.0,1.446919,0.000000,0.000000,0.000000,0.000000,0.000000,1.446919,0.0,0.000000,0.000000,0.000000,0.000000
강한,0.0,0.435318,0.0,0.435318,0.0,0.435318,0.435318,0.435318,0.000000,0.435318,0.435318,0.435318,0.0,0.000000,0.000000,0.435318,0.435318
거센,0.0,0.000000,0.0,0.887303,0.0,0.000000,0.000000,0.000000,0.887303,1.774606,0.887303,0.000000,0.0,0.000000,0.887303,0.887303,0.000000
거친,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.446919,2.893838,0.000000,0.000000,0.0,1.446919,0.000000,0.000000,0.000000


In [138]:
df.columns = ['액션', '코미디', '다큐멘터리', '판타지', '공포', '음악', '로맨스', '스포츠', '서부','애니메이션', '범죄', '드라마', '역사', '가족', '스릴러', 'SF', '전쟁']

In [139]:
for genre in df.columns:
    print('========= %s =========' % genre)
    print(df[genre].sort_values(ascending=False).head(5))
    print(df[df[genre]>0][genre].sort_values(ascending=False))

========= 액션 =========
안타까운    1.446919
무서운     1.223775
엄청난     1.044920
멋진      0.753772
어릴      0.753772
Name: 액션, dtype: float64
안타까운    1.446919
무서운     1.223775
엄청난     1.044920
멋진      0.753772
어릴      0.753772
어린      0.696613
낯선      0.194156
Name: 액션, dtype: float64
========= 코미디 =========
예쁜      2.447551
나쁜      2.140066
빛바랜     1.734601
아름다운    1.507544
감미로운    1.446919
Name: 코미디, dtype: float64
예쁜      2.447551
나쁜      2.140066
빛바랜     1.734601
아름다운    1.507544
감미로운    1.446919
뜨거운     1.223775
힘든      1.041454
뛰어난     0.753772
멋진      0.753772
어릴      0.753772
엄청난     0.696613
강한      0.435318
낯선      0.388312
어린      0.348307
Name: 코미디, dtype: float64
========= 다큐멘터리 =========
놀라운     3.469202
귀한      2.140066
급한      1.734601
촌스러운    1.734601
남다른     1.446919
Name: 다큐멘터리, dtype: float64
놀라운     3.469202
귀한      2.140066
급한      1.734601
촌스러운    1.734601
남다른     1.446919
커다란     1.223775
어린      0.696613
Name: 다큐멘터리, dtype: float64
========= 판타지 =========
무서운    2.44755